[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/elcursodeia-code/blob/master/dl/01_intro.ipynb)

# Tu primera red neuronal

In [1]:
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"].values.astype(np.float32), mnist["target"].values.astype(int)

In [2]:
from torch.nn import Sequential as S
from torch.nn import Linear as L
from torch.nn import ReLU as R

model = S(L(784,128),R(),L(128,10))

In [3]:
from tqdm import tqdm
import torch

X_train, X_test = torch.from_numpy(X[:60000] / 255.), torch.from_numpy(X[60000:] / 255.)
y_train, y_test = torch.from_numpy(y[:60000]), torch.from_numpy(y[60000:])
bs = 32
num_batches = len(X_train) // bs

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
	for b in tqdm(range(num_batches)):
		x = X_train[b*bs:(b+1)*bs]
		y = y_train[b*bs:(b+1)*bs]
		y_hat = model(x)
		loss = loss_fn(y_hat, y)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
	print(f"Epoch {epoch+1} loss: {loss.item():.3f}")

100%|██████████| 1875/1875 [00:01<00:00, 1304.44it/s]


Epoch 1 loss: 0.036


100%|██████████| 1875/1875 [00:01<00:00, 1251.26it/s]


Epoch 2 loss: 0.036


100%|██████████| 1875/1875 [00:01<00:00, 1249.70it/s]


...

Epoch 8 loss: 0.014


100%|██████████| 1875/1875 [00:01<00:00, 1229.85it/s]


Epoch 9 loss: 0.005


100%|██████████| 1875/1875 [00:01<00:00, 1215.29it/s]

Epoch 10 loss: 0.001


In [4]:
def evaluate(model, X_test, y_test):
	acc = 0
	with torch.no_grad():
		for b in range(num_batches):
			x = X_test[b*bs:(b+1)*bs]
			y = y_test[b*bs:(b+1)*bs]
			y_hat = model(x)
			acc += torch.sum(torch.argmax(y_hat, dim=1) == y).item()
	return acc

acc = evaluate(model, X_test, y_test)
print(f"Accuracy: {acc} / {len(X_test)}")

Accuracy: 9758 / 10000


In [6]:
torch.onnx.export(
    model.cpu(),
    torch.randn(10, 784),
    'mnist-app/model.onnx',
    opset_version=11,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}
})